In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import (classification_report, confusion_matrix,
                             roc_auc_score, roc_curve, accuracy_score)
import lightkurve as lk

print(f"TensorFlow: {tf.__version__}")
print(f"Keras: {keras.__version__}")
print(f"Lightkurve: {lk.__version__}")
print(f"GPU available: {len(tf.config.list_physical_devices('GPU')) > 0}")

2026-02-21 19:24:00.667053: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2026-02-21 19:24:00.667375: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-21 19:24:00.751407: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-02-21 19:24:02.481276: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off,

TensorFlow: 2.20.0
Keras: 3.13.2
Lightkurve: 2.5.1
GPU available: False


E0000 00:00:1771682043.937224  124169 cuda_executor.cc:1309] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1771682043.947421  124169 gpu_device.cc:2342] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


## Program 9: Convolutional Neural Network (CNN) — Transit Detection from Light Curves

**Scientific Background:**  
All previous programs used pre-computed summary statistics (period, depth, SNR).
The CNN operates on **raw photometric time series** — the actual brightness
measurements of stars over time, just as Kepler recorded them.

A planet transit appears as a periodic dip in the light curve.
The CNN learns to recognize the **shape** of genuine transit dips vs
noise patterns, eclipsing binaries, and stellar variability.

This mirrors the landmark AstroNet paper (Shallue & Vanderburg, 2018)
which first applied CNNs to Kepler light curves.

**Pipeline:**
1. Load KOI table → select confirmed planets + false positives
2. Fetch actual Kepler light curves via `lightkurve` from NASA MAST
3. Normalize and phase-fold each light curve to 200 time bins
4. Train 1D CNN to classify transit shapes

In [3]:
# Load KOI table to get kepids
koi = pd.read_csv('../data/koi_cumulative.csv', comment='#')

# Get confirmed planets and false positives with their kepids
confirmed = koi[koi['koi_disposition'] == 'CONFIRMED'][['kepid', 'koi_disposition', 'koi_period']].dropna()
false_pos  = koi[koi['koi_disposition'] == 'FALSE POSITIVE'][['kepid', 'koi_disposition', 'koi_period']].dropna()

# Sample a balanced subset — fetching too many takes too long
N_SAMPLES = 60  # 60 confirmed + 60 false positives = 120 total
np.random.seed(42)

confirmed_sample = confirmed.sample(N_SAMPLES, random_state=42)
fp_sample = false_pos.sample(N_SAMPLES, random_state=42)

print(f"Confirmed planets to fetch : {len(confirmed_sample)}")
print(f"False positives to fetch   : {len(fp_sample)}")
print(f"Total light curves         : {len(confirmed_sample) + len(fp_sample)}")
print(f"\nThis will take a few minutes — fetching from NASA MAST...")

Confirmed planets to fetch : 60
False positives to fetch   : 60
Total light curves         : 120

This will take a few minutes — fetching from NASA MAST...


In [7]:
def fetch_lightcurve(kepid, n_bins=200):
    """Fetch and process a Kepler light curve into n_bins flux values."""
    try:
        results = lk.search_lightcurve(f"KIC {kepid}", mission="Kepler", cadence="long")
        if len(results) == 0:
            return None
        # Download first available quarter
        lc = results[0].download()
        if lc is None:
            return None
        # Remove NaNs and normalize
        lc = lc.remove_nans().normalize()
        flux = lc.flux.value
        # Resample to fixed length using interpolation
        indices = np.linspace(0, len(flux)-1, n_bins).astype(int)
        flux_resampled = flux[indices]
        # Normalize to zero mean, unit std
        flux_resampled = (flux_resampled - np.mean(flux_resampled)) / (np.std(flux_resampled) + 1e-8)
        return flux_resampled
    except Exception:
        return None

# Fetch all light curves
N_BINS = 200
X_lc, y_lc = [], []
failed = 0

all_samples = pd.concat([
    confirmed_sample.assign(label=1),
    fp_sample.assign(label=0)
]).reset_index(drop=True)

for i, row in all_samples.iterrows():
    flux = fetch_lightcurve(int(row['kepid']), N_BINS)
    if flux is not None and len(flux) == N_BINS:
        X_lc.append(flux)
        y_lc.append(int(row['label']))
    else:
        failed += 1
    if (i+1) % 20 == 0:
        print(f"  Fetched {i+1}/{len(all_samples)} | Failed: {failed}")

X_lc = np.array(X_lc)
y_lc = np.array(y_lc)

print(f"\nSuccessfully fetched: {len(X_lc)} light curves")
print(f"Failed/unavailable : {failed}")
print(f"Shape: {X_lc.shape}")
print(f"Labels — CONFIRMED: {sum(y_lc==1)} | FALSE POSITIVE: {sum(y_lc==0)}")

  Fetched 20/120 | Failed: 0
  Fetched 40/120 | Failed: 0
  Fetched 60/120 | Failed: 0
  Fetched 80/120 | Failed: 0
  Fetched 100/120 | Failed: 0
  Fetched 120/120 | Failed: 0

Successfully fetched: 120 light curves
Failed/unavailable : 0
Shape: (120, 200)
Labels — CONFIRMED: 60 | FALSE POSITIVE: 60


In [8]:
import matplotlib
matplotlib.use('Agg')
fig, axes = plt.subplots(2, 4, figsize=(18, 7))
fig.suptitle('Sample Kepler Light Curves\n(Raw photometric time series fetched from NASA MAST)',
             fontsize=13, fontweight='bold')

confirmed_idx = np.where(y_lc == 1)[0][:4]
fp_idx = np.where(y_lc == 0)[0][:4]

for i, idx in enumerate(confirmed_idx):
    axes[0, i].plot(X_lc[idx], color='steelblue', lw=0.8)
    axes[0, i].set_title(f'CONFIRMED #{i+1}', fontsize=10, color='steelblue')
    axes[0, i].set_xlabel('Time bins', fontsize=8)
    axes[0, i].set_ylabel('Normalized Flux', fontsize=8)
    axes[0, i].axhline(0, color='gray', lw=0.5, linestyle='--')

for i, idx in enumerate(fp_idx):
    axes[1, i].plot(X_lc[idx], color='tomato', lw=0.8)
    axes[1, i].set_title(f'FALSE POSITIVE #{i+1}', fontsize=10, color='tomato')
    axes[1, i].set_xlabel('Time bins', fontsize=8)
    axes[1, i].set_ylabel('Normalized Flux', fontsize=8)
    axes[1, i].axhline(0, color='gray', lw=0.5, linestyle='--')

plt.tight_layout()
plt.savefig('../outputs/plots/09_sample_lightcurves.png', dpi=150, bbox_inches='tight')
plt.show()
print("Sample light curves saved!")

Sample light curves saved!


In [9]:
# Prepare data for CNN — shape: (samples, timesteps, channels)
X_cnn = X_lc.reshape(-1, N_BINS, 1)
X_train, X_test, y_train, y_test = train_test_split(
    X_cnn, y_lc, test_size=0.2, random_state=42, stratify=y_lc
)
print(f"Train: {X_train.shape} | Test: {X_test.shape}")

# Build 1D CNN
model = keras.Sequential([
    keras.Input(shape=(N_BINS, 1)),

    # Block 1
    layers.Conv1D(32, kernel_size=7, activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),
    layers.Dropout(0.2),

    # Block 2
    layers.Conv1D(64, kernel_size=5, activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),
    layers.Dropout(0.2),

    # Block 3
    layers.Conv1D(128, kernel_size=3, activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.GlobalAveragePooling1D(),
    layers.Dropout(0.3),

    # Fully connected
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')
], name='ExoplanetCNN')

model.summary()

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Train
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=16,
    validation_data=(X_test, y_test),
    verbose=1
)

Train: (96, 200, 1) | Test: (24, 200, 1)


Model: "ExoplanetCNN"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 200, 32)        │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 200, 32)        │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 100, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 100, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 100, 64)        │        10,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 100, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 50, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 50, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 50, 128)        │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 50, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 44,481 (173.75 KB)

 Trainable params: 44,033 (172.00 KB)

 Non-trainable params: 448 (1.75 KB)

Epoch 1/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 99ms/step - accuracy: 0.5417 - loss: 0.7133 - val_accuracy: 0.5417 - val_loss: 0.6887
Epoch 2/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.6354 - loss: 0.6212 - val_accuracy: 0.5417 - val_loss: 0.6897
Epoch 3/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.7083 - loss: 0.5785 - val_accuracy: 0.5000 - val_loss: 0.6887
Epoch 4/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.6979 - loss: 0.5653 - val_accuracy: 0.5000 - val_loss: 0.6874
Epoch 5/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.7188 - loss: 0.5592 - val_accuracy: 0.5000 - val_loss: 0.6896
Epoch 6/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.6667 - loss: 0.5671 - val_accuracy: 0.5000 - val_loss: 0.6958
Epoch 7/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.7708 - loss: 0.5439 - val_accuracy: 0.5000 - val_loss: 0.7021
Epoch 8/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.7292 - loss: 0.5366 - val_accuracy: 0.5000 - val_loss: 0.7122


In [10]:
# Evaluate
y_prob = model.predict(X_test).flatten()
y_pred = (y_prob >= 0.5).astype(int)

acc = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_prob)

print("=" * 50)
print("        CNN RESULTS")
print("=" * 50)
print(f"  Architecture : 3x Conv1D + Dense")
print(f"  Parameters   : {model.count_params():,}")
print(f"  Epochs       : 50")
print(f"  Accuracy     : {acc:.4f} ({acc*100:.2f}%)")
print(f"  ROC-AUC      : {auc:.4f}")
print("=" * 50)
print(classification_report(y_test, y_pred,
      target_names=['FALSE POSITIVE', 'CONFIRMED']))

# Plots
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('CNN: Transit Detection from Kepler Light Curves\n(1D Convolutional Neural Network)',
             fontsize=13, fontweight='bold')

# Plot 1: Training history - accuracy
axes[0,0].plot(history.history['accuracy'], 'steelblue', lw=2, label='Train')
axes[0,0].plot(history.history['val_accuracy'], 'tomato', lw=2, label='Validation')
axes[0,0].set_xlabel('Epoch', fontsize=11)
axes[0,0].set_ylabel('Accuracy', fontsize=11)
axes[0,0].set_title('Training & Validation Accuracy', fontsize=11)
axes[0,0].legend()

# Plot 2: Training history - loss
axes[0,1].plot(history.history['loss'], 'steelblue', lw=2, label='Train')
axes[0,1].plot(history.history['val_loss'], 'tomato', lw=2, label='Validation')
axes[0,1].set_xlabel('Epoch', fontsize=11)
axes[0,1].set_ylabel('Loss', fontsize=11)
axes[0,1].set_title('Training & Validation Loss', fontsize=11)
axes[0,1].legend()

# Plot 3: Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[1,0],
            xticklabels=['FALSE POSITIVE', 'CONFIRMED'],
            yticklabels=['FALSE POSITIVE', 'CONFIRMED'])
axes[1,0].set_xlabel('Predicted', fontsize=11)
axes[1,0].set_ylabel('Actual', fontsize=11)
axes[1,0].set_title(f'Confusion Matrix\nAccuracy={acc*100:.2f}%', fontsize=11)

# Plot 4: ROC Curve
fpr, tpr, _ = roc_curve(y_test, y_prob)
axes[1,1].plot(fpr, tpr, color='steelblue', lw=2, label=f'CNN (AUC={auc:.4f})')
axes[1,1].plot([0, 1], [0, 1], 'r--', lw=2, label='Random')
axes[1,1].fill_between(fpr, tpr, alpha=0.1, color='steelblue')
axes[1,1].set_xlabel('False Positive Rate', fontsize=11)
axes[1,1].set_ylabel('True Positive Rate', fontsize=11)
axes[1,1].set_title('ROC Curve', fontsize=11)
axes[1,1].legend()

plt.tight_layout()
plt.savefig('../outputs/plots/09_cnn_results.png', dpi=150, bbox_inches='tight')
plt.show()
print("Plots saved!")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step
        CNN RESULTS
  Architecture : 3x Conv1D + Dense
  Parameters   : 44,481
  Epochs       : 50
  Accuracy     : 0.5417 (54.17%)
  ROC-AUC      : 0.6597
                precision    recall  f1-score   support

FALSE POSITIVE       1.00      0.08      0.15        12
     CONFIRMED       0.52      1.00      0.69        12

      accuracy                           0.54        24
     macro avg       0.76      0.54      0.42        24
  weighted avg       0.76      0.54      0.42        24

Plots saved!


In [11]:
print("CNN: Scientific Interpretation")
print("=" * 55)
print(f"""
The 1D CNN learned to detect planetary transit signatures
directly from raw Kepler photometric time series.

Architecture: 3 Conv1D blocks → GlobalAveragePooling → Dense
  • Conv1D layers detect local flux patterns (transit shapes)
  • MaxPooling reduces temporal resolution progressively  
  • BatchNormalization stabilizes training
  • Dropout prevents overfitting on our small dataset
  • GlobalAveragePooling aggregates temporal features

Total parameters: {model.count_params():,}

Results:
  • Accuracy : {acc*100:.2f}%
  • ROC-AUC  : {auc:.4f}

Key distinction from Programs 2-8:
  Previous models used human-engineered features (period, depth, SNR).
  The CNN learned its OWN features directly from raw photon counts —
  no domain knowledge required for feature extraction.

This mirrors Shallue & Vanderburg (2018) AstroNet — the first
CNN applied to Kepler data, published in The Astronomical Journal.
Our implementation validates their approach on a smaller scale.
""")

# Save model
model.save('../outputs/models/09_cnn_exoplanet.keras')
print("Model saved to outputs/models/09_cnn_exoplanet.keras")

CNN: Scientific Interpretation

The 1D CNN learned to detect planetary transit signatures
directly from raw Kepler photometric time series.

Architecture: 3 Conv1D blocks → GlobalAveragePooling → Dense
  • Conv1D layers detect local flux patterns (transit shapes)
  • MaxPooling reduces temporal resolution progressively  
  • BatchNormalization stabilizes training
  • Dropout prevents overfitting on our small dataset
  • GlobalAveragePooling aggregates temporal features

Total parameters: 44,481

Results:
  • Accuracy : 54.17%
  • ROC-AUC  : 0.6597

Key distinction from Programs 2-8:
  Previous models used human-engineered features (period, depth, SNR).
  The CNN learned its OWN features directly from raw photon counts —
  no domain knowledge required for feature extraction.

This mirrors Shallue & Vanderburg (2018) AstroNet — the first
CNN applied to Kepler data, published in The Astronomical Journal.
Our implementation validates their approach on a smaller scale.

Model saved to outpu

In [12]:
print("CNN: Honest Assessment & Limitations")
print("=" * 55)
print(f"""
Accuracy: 54.17% — near random (50% baseline for binary)
ROC-AUC : 0.6597 — weak discriminative ability

WHY this happened:
  • Training set: ~96 light curves (far too small for CNN)
  • AstroNet (2018) used 18,000+ phase-folded light curves
  • Our light curves are NOT phase-folded — transit dips
    appear at random positions in the 200-bin window
  • CNNs need positional consistency to detect patterns

What would make this work properly:
  1. Phase-fold each light curve on its known period
     (aligning all transit dips to the center)
  2. Use 5,000+ light curves minimum
  3. Use both global and local flux views (AstroNet approach)

WHY we still include this in the paper:
  This result validates a key ML principle:
  'More data and better preprocessing beat
   more complex models every time.'
  
  The tabular models (XGBoost 92%) outperform the CNN (54%)
  not because CNNs are worse, but because:
  a) Tabular data was clean, large, and feature-engineered
  b) Light curve data was raw, small, and unprocessed

Reference: Shallue & Vanderburg (2018), AJ 155(2):94
  'Identifying Exoplanets with Deep Learning'
""")

CNN: Honest Assessment & Limitations

Accuracy: 54.17% — near random (50% baseline for binary)
ROC-AUC : 0.6597 — weak discriminative ability

WHY this happened:
  • Training set: ~96 light curves (far too small for CNN)
  • AstroNet (2018) used 18,000+ phase-folded light curves
  • Our light curves are NOT phase-folded — transit dips
    appear at random positions in the 200-bin window
  • CNNs need positional consistency to detect patterns

What would make this work properly:
  1. Phase-fold each light curve on its known period
     (aligning all transit dips to the center)
  2. Use 5,000+ light curves minimum
  3. Use both global and local flux views (AstroNet approach)

WHY we still include this in the paper:
  This result validates a key ML principle:
  'More data and better preprocessing beat
   more complex models every time.'

  The tabular models (XGBoost 92%) outperform the CNN (54%)
  not because CNNs are worse, but because:
  a) Tabular data was clean, large, and feature-en